In [1]:
import os

In [2]:
%pwd


'/Users/selvintuscano/Desktop/Project/Kidney-Disease-Classification-using-Deep-Learning/research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'/Users/selvintuscano/Desktop/Project/Kidney-Disease-Classification-using-Deep-Learning'

In [5]:
import os

os.environ["MLFLOW_TRACKING_URI"] ="https://dagshub.com/selvintuscano31/Kidney-Disease-Classification-using-Deep-Learning.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] ="selvintuscano31"
os.environ["MLFLOW_TRACKING_PASSWORD"] ="97a22e9d2a17a4e9947051de36c9991f9cbc00b8"




In [ ]:
export MLFLOW_TRACKING_URI=https://dagshub.com/selvintuscano31/Kidney-Disease-Classification-using-Deep-Learning.mlflow

export MLFLOW_TRACKING_USERNAME=selvintuscano31

export MLFLOW_TRACKING_PASSWORD=97a22e9d2a17a4e9947051de36c9991f9cbc00b8

In [6]:
import tensorflow as tf


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


2025-07-30 15:15:23.723163: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-07-30 15:15:23.723192: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-30 15:15:23.723198: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-30 15:15:23.723228: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-30 15:15:23.723242: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/selvintuscano31/Kidney-Disease-Classification-using-Deep-Learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    import mlflow
    import mlflow.tensorflow
    from urllib.parse import urlparse

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        #mlflow.set_tracking_uri(self.config.mlflow_uri)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.set_experiment("my_experiment")
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
       
            import tensorflow.keras as keras
            if not hasattr(keras, "__version__"):
                keras.__version__ = tf.__version__

            if tracking_url_type_store != "file":
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.tensorflow.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-07-30 15:15:29,470: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-30 15:15:29,472: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-30 15:15:29,473: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.


2025-07-30 15:15:29.744124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 140ms/step - loss: 3.1039 - accuracy: 0.6043
[2025-07-30 15:15:31,131: INFO: common: json file saved at: scores.json]


2025/07/30 15:15:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/2k/2h026m417z1b3v_6p87mbf7c0000gn/T/tmpbxe7z5ux/model/data/model/assets
[2025-07-30 15:15:32,351: INFO: builder_impl: Assets written to: /var/folders/2k/2h026m417z1b3v_6p87mbf7c0000gn/T/tmpbxe7z5ux/model/data/model/assets]


2025/07/30 15:15:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/2k/2h026m417z1b3v_6p87mbf7c0000gn/T/tmpbxe7z5ux/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/07/30 15:15:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


In [18]:
print("Tracking URI:", mlflow.get_tracking_uri())
print("Registry URI:", mlflow.get_registry_uri())

Tracking URI: https://dagshub.com/selvintuscano31/Kidney-Disease-Classification-using-Deep-Learning.mlflow
Registry URI: https://dagshub.com/selvintuscano31/Kidney-Disease-Classification-using-Deep-Learning.mlflow


In [19]:
mlflow.set_experiment("my_experiment")


2025/07/30 12:43:05 INFO mlflow.tracking.fluent: Experiment with name 'my_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/421c5c3671524e98a0119b1e25cd6221', creation_time=1753893786035, experiment_id='1', last_update_time=1753893786035, lifecycle_stage='active', name='my_experiment', tags={}>